**Import**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import zipfile
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from PIL import Image
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skimage.metrics import structural_similarity as ssim
import cv2
from tqdm.auto import tqdm
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.datasets import ImageFolder
from pathlib import Path
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, f1_score
import seaborn as sns

In [ ]:
!git clone https://github.com/Mvryo02/AnomalyDetection_PlaneBlade.git

In [ ]:
def bilateral_filter(image):
    np_image = np.array(image)  # Convert PIL image to numpy array
    filtered = cv2.bilateralFilter(np_image, d=9, sigmaColor=75, sigmaSpace=75)
    return Image.fromarray(filtered)  # Convert back to PIL image

def sharpen(image):
    np_image = np.array(image)  # Convert PIL image to numpy array
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(np_image, -1, kernel)
    return Image.fromarray(sharpened)  # Convert back to PIL image

class CustomTransform:
    def __init__(self, additional_transform=None):
        self.additional_transform = additional_transform

    def __call__(self, img):
        img = bilateral_filter(img) 
        img = sharpen(img)  
        if self.additional_transform:
            img = self.additional_transform(img)
        return img

transform_train = transforms.Compose([
    CustomTransform(),                      # Apply costum filters
    transforms.RandomHorizontalFlip(p=0.5), # random horizontal flip
    transforms.RandomRotation(15),          # random rotation of 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation = 0.2), 
    transforms.Resize((224,224)),          
    transforms.ToTensor(),                # Convert image to tensor  
])

transform_test = transforms.Compose([
    CustomTransform(),  
    transforms.Resize((224,224)),
    transforms.ToTensor(),

])

In [ ]:
def decision_function(segm_map):  

    mean_top_10_values = []

    for map in segm_map:
        # Flatten the tensor
        flattened_tensor = map.reshape(-1)

        # Sort the flattened tensor along the feature dimension (descending order)
        sorted_tensor, _ = torch.sort(flattened_tensor,descending=True)

        # Take the top 10 values along the feature dimension
        mean_top_10_value = sorted_tensor[:10].mean()

        mean_top_10_values.append(mean_top_10_value)

    return torch.stack(mean_top_10_values)

**Method 1: Autoencoder**

In [ ]:
#model 1 
class Autoencoder1(nn.Module):
    def __init__(self):
        super(Autoencoder1, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(True),        
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1), 
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), 
            nn.ReLU(True)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1), 
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),   
            nn.Sigmoid()   
        )


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
#Model 2
class Autoencoder2(nn.Module):
    def __init__(self):
        super(Autoencoder2, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(64, 1, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.Sigmoid() 
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
#Model 3
class Autoencoder3(nn.Module):
    def __init__(self):
        super(Autoencoder3, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1), 
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.Sigmoid()  # Mappare i valori tra [0,1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


**Method 1 : Autoencoder - Training**

In [ ]:
train_image_path = Path('/kaggle/working/AnomalyDetection_PlaneBlade-/train_detector')

good_dataset = ImageFolder(root=train_image_path, transform=transform_train)
train_dataset, validation_dataset = torch.utils.data.random_split(good_dataset, [0.8, 0.2])

# Set the batch size
BS = 32

# Create data loaders for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BS, shuffle=True)

In [ ]:
model = Autoencoder3()
model.cuda()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
Loss = []
Validation_Loss = []


def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=15):
    for epoch in range(epochs):
        model.train()  # Modalità di training
        train_loss = 0.0
        for images, _ in train_loader:
            images = images.to('cuda')

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, images)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        Loss.append(train)
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to('cuda')
                outputs = model(images)
                loss = criterion(outputs, images)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}, Training Loss: {train_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}")
        Loss.append(train_loss)
        Validation_Loss.append(val_loss)


train_model(model, train_loader, val_loader, criterion, optimizer, epochs=15)
plt.plot(Loss, label='Training Loss')
plt.plot(Validation_Loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.eval()

RECON_ERROR=[]
for data,_ in train_loader:
    
    with torch.no_grad():
         recon = model(test_image.cuda())
    # Compute the loss
    recon_error =  ((features-recon)**2).mean(axis=(1))[:,3:-3,3:-3]
    # anomaly_score = segm_map.mean(axis=(1,2))
    
    RECON_ERROR.append(recon_error.cpu().unsqueeze(0))
    
RECON_ERROR = torch.cat(RECON_ERROR).cpu().numpy()

In [ ]:
possible_threshold = np.mean(RECON_ERROR) + 3 * np.std(RECON_ERROR)

heat_map_max, heat_map_min = np.max(RECON_ERROR), np.min(RECON_ERROR)

plt.hist(RECON_ERROR,bins=50)
plt.vlines(x=best_threshold,ymin=0,ymax=30,color='r')
plt.show()

**Method 1 : Autoencoder - Testing**

In [ ]:
y_true=[]
y_score=[]

model.eval()
from pathlib import Path



test_path = Path('/kaggle/working/AnomalyDetection_PlaneBlade/test_detector')

for path in test_path.glob('*/*.png'):
    fault_type = path.parts[-2]
    test_image = transform_test(Image.open(path)).cuda().unsqueeze(0)
    
    with torch.no_grad():
        
        # Forward pass
        recon = model(test_image)
    
    y_score_image = ((test_image - recon)**2).mean(axis=(1)).mean()

        
    y_true_image = 0 if fault_type == 'good' else 1
    
    y_true.append(y_true_image)
    y_score.append(y_score_image.cpu().numpy())
    
y_true = np.array(y_true)
y_score = np.array(y_score)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_score)


f1_scores = [f1_score(y_true, (y_score >= threshold).astype(int)) for threshold in thresholds]

# Select the best threshold based on F1 score
best_threshold = thresholds[np.argmax(f1_scores)]

#best_threshold=... to set the threshold to a wanted value

print(f'best_threshold = {best_threshold}')

auc_roc_score = roc_auc_score(y_true, (y_score >= best_threshold).astype(int))

print("AUC-ROC Score:", auc_roc_score)
precision = precision_score(y_true, (y_score >= best_threshold).astype(int))
recall = recall_score(y_true, (y_score >= best_threshold).astype(int))


print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_score)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc_roc_score)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

cm = confusion_matrix(y_true, (y_score >= best_threshold).astype(int))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['OK','NOK'])
disp.plot()
plt.show()

**Method 2: Autoencoder with a ResNet for feature extraction**

In [ ]:
class resnet_feature_extractor(torch.nn.Module):
    def __init__(self):
        """This class extracts the feature maps from a pretrained Resnet model."""
        super(resnet_feature_extractor, self).__init__()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)

        self.model.eval()
        for param in self.model.parameters():
            param.requires_grad = False

        

        # Hook to extract feature maps
        def hook(module, input, output) -> None:
            """This hook saves the extracted feature map on self.featured."""
            self.features.append(output)

        self.model.layer2[-1].register_forward_hook(hook)            
        self.model.layer3[-1].register_forward_hook(hook) 

    def forward(self, input):

        self.features = []
        with torch.no_grad():
            _ = self.model(input)

        self.avg = torch.nn.AvgPool2d(3, stride=1)
        fmap_size = self.features[0].shape[-2]         # Feature map sizes h, w
        self.resize = torch.nn.AdaptiveAvgPool2d(fmap_size)

        resized_maps = [self.resize(self.avg(fmap)) for fmap in self.features]
        patch = torch.cat(resized_maps, 1)            # Merge the resized feature maps

        return patch

In [ ]:
resnet_model = resnet50(weights=ResNet50_Weights.DEFAULT)
backbone = resnet_feature_extractor()

In [ ]:
#Model 1
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
    
       
        self.encoder = nn.Sequential(
            nn.Conv2d(1536, 868, kernel_size=1, stride=1),
            nn.BatchNorm2d(868),
            nn.ReLU(),
            nn.Dropout(0.3),  
            
            nn.Conv2d(868, 200, kernel_size=1, stride=1), 
            nn.BatchNorm2d(200),
            nn.ReLU(),
            nn.Dropout(0.3),  
            
            nn.Conv2d(200, 100, kernel_size=1, stride=1),  
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=1, stride=1), 
            nn.BatchNorm2d(200),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Conv2d(200, 868, kernel_size=1, stride=1),  
            nn.BatchNorm2d(868),
            nn.ReLU(),
            nn.Dropout(0.3),  
            
            nn.Conv2d(868, 1536, kernel_size=1, stride=1),  
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
#Model 2
class AutoEncoder2(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
    
       
        self.encoder = nn.Sequential(
            nn.Conv2d(1536, 868, kernel_size=1, stride=1),
            nn.BatchNorm2d(868),
            nn.ReLU(),
            
            nn.Conv2d(868, 200, kernel_size=1, stride=1), 
            nn.BatchNorm2d(200),
            nn.ReLU(), 
            
            nn.Conv2d(200, 100, kernel_size=1, stride=1),  
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=1, stride=1), 
            nn.BatchNorm2d(200),
            nn.ReLU(),
            
            nn.Conv2d(200, 868, kernel_size=1, stride=1),  
            nn.BatchNorm2d(868),
            nn.ReLU(), 
            
            nn.Conv2d(868, 1536, kernel_size=1, stride=1),  
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
#Model 3
class AutoEncoder3(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
    
       
        self.encoder = nn.Sequential(
            nn.Conv2d(1536, 868, kernel_size=1, stride=1),
            nn.BatchNorm2d(868),
            nn.ReLU(),
              
            
            nn.Conv2d(868, 400, kernel_size=1, stride=1), 
            nn.BatchNorm2d(400),
            nn.ReLU(),

            nn.Conv2d(400, 200, kernel_size=1, stride=1), 
            nn.BatchNorm2d(200),
            nn.ReLU(),
             
            
            nn.Conv2d(200, 100, kernel_size=1, stride=1),  
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=1, stride=1), 
            nn.BatchNorm2d(200),
            nn.ReLU(),
            
            nn.Conv2d(200, 400, kernel_size=1, stride=1),  
            nn.BatchNorm2d(400),
            nn.ReLU(),

            nn.Conv2d(400, 868, kernel_size=1, stride=1),  
            nn.BatchNorm2d(868),
            nn.ReLU(),
        
            
            nn.Conv2d(868, 1536, kernel_size=1, stride=1),  
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

**Method 2: Autoencoder with a ResNet for feature extraction - Training**

In [ ]:
model = AutoEncoder(in_channels=1536, latent_dim=100).cuda()
backbone.cuda()
model.cuda()
# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
Loss = []  
Validation_Loss = []  
num_epochs = 15
for epoch in tqdm(range(num_epochs)):
    model.train()
    for data,_ in train_loader:
        with torch.no_grad():
            features = backbone(data.cuda())
        # Forward pass
        output = model(features)
        # Compute the loss
        loss = criterion(output, features)
        # Backpropagation and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    Loss.append(loss.item())

    
    model.eval()  
    with torch.no_grad():
        val_loss_sum = 0.0
        num_batches = 0
        for data, _ in validation_loader:
            features = backbone(data.cuda())
            output = model(features)
            val_loss = criterion(output, features)
            val_loss_sum += val_loss.item()
            num_batches += 1
        val_loss_avg = val_loss_sum / num_batches
        Validation_Loss.append(val_loss_avg)

    
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss_avg:.4f}')


plt.plot(Loss, label='Training Loss')
plt.plot(Validation_Loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
# plt.show()

In [ ]:
model.eval()

RECON_ERROR=[]
for data,_ in train_loader:
    
    with torch.no_grad():
        features = backbone(data.cuda()).squeeze()
        # Forward pass
        recon = model(features)
    # Compute the loss
    segm_map =  ((features-recon)**2).mean(axis=(1))[:,3:-3,3:-3]
    anomaly_score = decision_function(segm_map)
    # anomaly_score = segm_map.mean(axis=(1,2))
    
    RECON_ERROR.append(anomaly_score)
    
RECON_ERROR = torch.cat(RECON_ERROR).cpu().numpy()

In [ ]:
possible_threshold = np.mean(RECON_ERROR) + 3 * np.std(RECON_ERROR)

heat_map_max, heat_map_min = np.max(RECON_ERROR), np.min(RECON_ERROR)

plt.hist(RECON_ERROR,bins=50)
plt.vlines(x=best_threshold,ymin=0,ymax=30,color='r')
plt.show()

**Method 2: Autoencoder with a ResNet for feature extraction - Testing**

In [ ]:
y_true=[]
y_score=[]

model.eval()
backbone.eval()


test_path = Path('/kaggle/working/AnomalyDetection_PlaneBlade/test_detector')

for path in test_path.glob('*/*.png'):
    fault_type = path.parts[-2]
    test_image = transform_test(Image.open(path)).cuda().unsqueeze(0)
    
    with torch.no_grad():
        features = backbone(test_image)
        # Forward pass
        recon = model(features)
    
    segm_map = ((features - recon)**2).mean(axis=(1))[:,8:-8,8:-8]
    y_score_image = decision_function(segm_map=segm_map)
    
    y_true_image = 0 if fault_type == 'good' else 1
    
    y_true.append(y_true_image)
    y_score.append(y_score_image.cpu().numpy())
    
y_true = np.array(y_true)
y_score = np.array(y_score)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_score)

# Calcolo dei F1 scores per ogni threshold
f1_scores = [f1_score(y_true, (y_score >= threshold).astype(int)) for threshold in thresholds]

# Select the best threshold based on F1 score
best_threshold = thresholds[np.argmax(f1_scores)]

#best_threshold=...

print(f'best_threshold = {best_threshold}')

auc_roc_score = roc_auc_score(y_true, (y_score >= best_threshold).astype(int))

print("AUC-ROC Score:", auc_roc_score)

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_score)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc_roc_score)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

cm = confusion_matrix(y_true, (y_score >= best_threshold).astype(int))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['OK','NOK'])
disp.plot()
plt.show()

**Classifier: Models**

In [ ]:
#CLassifier 1
class Classifier1(nn.Module):
    def __init__(self, num_classes):
        super(Classifier1, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 26 * 26, 128),  
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)  
        x = torch.flatten(x, 1) 
        x = self.fc_layers(x)  
        return x

In [ ]:
#CLassifier 2
class Classifier2(nn.Module):
    def __init__(self, num_classes):
        super(Classifier2, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 26 * 26, 128),  
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)  
        x = torch.flatten(x, 1)  
        x = self.fc_layers(x) 
        return x

In [ ]:
#Classifier 3
class Classifier3(nn.Module):
    def __init__(self, num_classes):
        super(Classifier3, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 26 * 26, 128),  
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)  
        x = torch.flatten(x, 1)  
        x = self.fc_layers(x) 
        return x

**Classifier: Training**

In [ ]:
num_classes = 4  #ablation, groove, facture, breakdown


classifier  = Classifier3(num_classes).cuda()
classifier.eval()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

In [ ]:
train2_image_path = Path('/kaggle/working/AnomalyDetection_PlaneBlade/train_classifier')

dataset = ImageFolder(root=train2_image_path, transform=transform)
train2_dataset, validation_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
# Set the batch size
BS = 32

# Create data loaders for training and testing datasets
classifier_train_loader = DataLoader(train2_dataset, batch_size=BS, shuffle=True)
classifier_validation_loader = DataLoader(validation_dataset, batch_size=BS, shuffle=True)

In [ ]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train() 
        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda() 
            optimizer.zero_grad()  
            outputs = model(images)  
            loss = criterion(outputs, labels)  
            loss.backward()  
            optimizer.step()  

        # Validazione
        model.eval()  
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.cuda(), labels.cuda()  
                outputs = model(images)  
                _, predicted = torch.max(outputs.data, 1) 
                total += labels.size(0) 
                correct += (predicted == labels).sum().item()  

        print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {100 * correct / total:.2f}%')

In [ ]:
num_epochs = 25
train_and_validate(classifier, classifier_train_loader, classifier_validation_loader, criterion, optimizer, num_epochs)

**Classifier: Testing**

In [ ]:
test_image_path = Path('/kaggle/working/AnomalyDetection_PlaneBlade/test_classifier')
testset=ImageFolder(root=train2_image_path, transform=transform)
classifier_test_loader = DataLoader(testset, batch_size=BS, shuffle=False)

In [ ]:
def test_model(model, test_loader):
    model.eval()  # Imposta il modello in modalità valutazione (disattiva dropout, batchnorm, ecc.)
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():  # Disattiva il calcolo del gradiente
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()  # Sposta su GPU se disponibile
            outputs = model(images)  # Passa le immagini attraverso il modello
            _, predicted = torch.max(outputs.data, 1)  # Ottieni le predizioni con il massimo logit
            total += labels.size(0)  # Incrementa il numero totale di campioni
            correct += (predicted == labels).sum().item()  # Conta le predizioni corrette
            all_predictions.extend(predicted.cpu().numpy())  # Salva le predizioni
            all_labels.extend(labels.cpu().numpy())  # Salva le etichette reali

    accuracy = 100 * correct / total  # Calcola l'accuratezza
    print(f'Test Accuracy: {accuracy:.2f}%')

    return all_predictions, all_labels

In [ ]:
predictions, labels = test_model(classifier, classifier_test_loader)
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Classification Report
print(classification_report(labels, predictions))


conf_matrix = confusion_matrix(labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()